In [1]:
from pinecone import Pinecone as Pine
import pandas as pd
import time

from gemini_client import Gemini_client as Gemini

# List of Prompts to send for the Experiments

We are using 3 for variation

In [2]:
prompts = [
    """Based on the provided customer feedback, generate a single-paragraph summary of the overall sentiment 
    regarding the iPhone 16. Identify the single most frequently mentioned positive attribute and the 
    single most frequent complaint mentioned about the device in the data. Be sure both points are directly supported 
    by the feedback.""",
    
    """Generate a report structured as a pro-and-con list specifically comparing the iPhone 16 vs. the Samsung S24. 
    The report must include three distinct product areas (e.g., Battery Life, Design, Performance) where customers 
    clearly prefer the iPhone and three distinct product areas where customers clearly prefer the Samsung. You must 
    source all six points entirely from the feedback.""",
    
    """Using the provided customer feedback, identify which of the two devices — iPhone 16 or Samsung S24 — is currently 
    perceived as weaker in overall satisfaction. Then, propose a **single, evidence-backed product or experience change** 
    that would most directly address the top recurring customer pain point for that device. Your proposal must cite 
    specific feedback patterns or quotes from the data to justify the recommendation."""
]

In [3]:
gemini = Gemini()

# Experiment 1: Generating output on pre-trained LLM's without augmentation

In [ ]:
data = []

for prompt in prompts:
    try:
        response = gemini.generate_content(prompt)
    
        data.append({
            "Prompt": prompt,
            "Response": response
        })
        
    except Exception as e:
        print(f"Exception found: {e}")
        time.sleep(2)

In [ ]:
df = pd.DataFrame(data)

with pd.ExcelWriter(r"Prompt_responses.xlsx", mode="a", engine="openpyxl", if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name="Raw_run")

# Required Variables for Experiments 2 and 3

In [4]:
comments_df = pd.read_excel(r"labeled_comments.xls")

len(comments_df)

1170

In [5]:
batches = {}
batch_size = 20

for i, row in comments_df.iterrows():
    if type(row["comment_text"]) != str:
        row["comment_text"] = str(row["comment_text"])
    
    batch_id = (i // batch_size) + 1
    key = f"batch_{batch_id}"
    batches.setdefault(key, "")
    batches[key] += row["comment_text"] + "\n"

In [6]:
type(batches)

dict

# Experiment 2: Augmentation of Data in Raw format

In [ ]:
import time
import json

batches_list = list(batches.items())
retry_delay = 5          # initial sleep (seconds)
max_delay = 300          # max backoff (5 min)
checkpoint_file = "checkpoint.json"

# --- Load last checkpoint if exists
try:
    with open(checkpoint_file, "r") as f:
        start_index = json.load(f).get("last_index", 0)
except FileNotFoundError:
    start_index = 0

print(f"Resuming from batch index {start_index}")

for idx in range(start_index, len(batches_list)):
    batch_name, prompt = batches_list[idx]

    while True:
        try:
            response = gemini.responses_experiment_2(prompt)

            print(f"✅ Completed {batch_name}")

            # Reset delay after success
            retry_delay = 5

            # Save checkpoint
            with open(checkpoint_file, "w") as f:
                json.dump({"last_index": idx + 1}, f)

            # Respect pacing
            time.sleep(5)
            break  # proceed to next batch

        except Exception as e:
            if "503" in str(e):
                print(f"⚠️ Model overloaded at {batch_name}. Retrying in {retry_delay}s...")
                time.sleep(retry_delay)
                retry_delay = min(retry_delay * 2, max_delay)  # exponential backoff
            else:
                print(f"❌ Unexpected error at {batch_name}: {e}")
                raise


Resuming from batch index 1
✅ Completed batch_2
✅ Completed batch_3
✅ Completed batch_4
✅ Completed batch_5
✅ Completed batch_6
⚠️ Model overloaded at batch_7. Retrying in 5s...
⚠️ Model overloaded at batch_7. Retrying in 10s...
⚠️ Model overloaded at batch_7. Retrying in 20s...
✅ Completed batch_7
✅ Completed batch_8
✅ Completed batch_9
✅ Completed batch_10
✅ Completed batch_11
⚠️ Model overloaded at batch_12. Retrying in 5s...
⚠️ Model overloaded at batch_12. Retrying in 10s...
✅ Completed batch_12
✅ Completed batch_13
✅ Completed batch_14
✅ Completed batch_15
⚠️ Model overloaded at batch_16. Retrying in 5s...
⚠️ Model overloaded at batch_16. Retrying in 10s...
✅ Completed batch_16
✅ Completed batch_17
⚠️ Model overloaded at batch_18. Retrying in 5s...


In [ ]:
data = []

for prompt in prompts:
    payload = {}
    response = gemini.responses_experiment_2(prompt)
    payload[prompt] = response
    data.append(payload)

In [ ]:
df = pd.DataFrame(data)

with pd.ExcelWriter(r"Prompt_responses.xlsx", engine="openpyxl", mode='a') as writer:
    df.to_excel(writer, "Raw_Message_Augmentation")

# Experiment 3: RAG Pipeline

In [7]:
from pine_client import PineConeDB

pine = PineConeDB()

In [8]:
for batch_name, batch_text in batches.items():
    record = [{
        "id": batch_name,
        "chunk_text": batch_text   # field Pinecone will embed automatically
    }]
    pine.upsert_data(record)

Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading records......
Uploaded
Uploading 

In [9]:
data = []

for prompt in prompts:
    # --- Retrieve context from Pinecone
    result = pine.search_index(prompt)

    # defensive parsing to avoid key errors
    hits = result.get("result", {}).get("hits", [])

    relevant_texts = []
    for hit in hits:
        # handle any malformed hit objects
        chunk_text = hit.get("fields", {}).get("chunk_text", None)
        if chunk_text:
            relevant_texts.append(chunk_text.strip())

    # join top-k results into a context block
    final_text = "\n\n".join(relevant_texts)
    if not final_text:
        final_text = "[No relevant context retrieved.]"

    # --- Construct context-aware prompt
    final_prompt = (
        f"Context retrieved from indexed customer feedback:\n{final_text}\n\n"
        f"Task:\n{prompt}"
    )

    # --- Generate model response
    try:
        response = gemini.generate_content(final_prompt)
        # extract plain text response if wrapped
        response_text = (
            response.text
            if hasattr(response, "text")
            else str(response)
        )
    except Exception as e:
        print(f"⚠️ Error for prompt: {prompt[:60]}... -> {e}")
        response_text = f"ERROR: {e}"

    # --- Store result
    data.append({
        "Prompt": prompt,
        "Retrieved_Context": final_text,
        "Response": response_text
    })
    
    time.sleep(15)

⚠️ Error for prompt: Using the provided customer feedback, identify which of the ... -> 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}


In [10]:
# --- Convert to DataFrame
df = pd.DataFrame(data)

# save to excel
with pd.ExcelWriter(r"Prompt_responses.xlsx", mode="a", engine="openpyxl", if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name="rag_experiment")

In [12]:
# last prompt failed
prompt = prompts[2]

result = pine.search_index(prompt)

# defensive parsing to avoid key errors
hits = result.get("result", {}).get("hits", [])
relevant_texts = []

for hit in hits:
    # handle any malformed hit objects
    chunk_text = hit.get("fields", {}).get("chunk_text", None)
    if chunk_text:
        relevant_texts.append(chunk_text.strip())

# join top-k results into a context block
final_text = "\n\n".join(relevant_texts)

if not final_text:
    final_text = "[No relevant context retrieved.]"
# --- Construct context-aware prompt
final_prompt = (
    f"Context retrieved from indexed customer feedback:\n{final_text}\n\n"
    f"Task:\n{prompt}"
)

# --- Generate model response
try:
    response = gemini.generate_content(final_prompt)
    # extract plain text response if wrapped
    response_text = (
        response.text
        if hasattr(response, "text")
        else str(response)
    )
    
except Exception as e:
    print(f"⚠️ Error for prompt: {prompt[:60]}... -> {e}")
    response_text = f"ERROR: {e}"
    
# --- Store result
data.append({
    "Prompt": prompt,
    "Retrieved_Context": final_text,
    "Response": response_text
})

In [13]:
response_text

'Based on the provided customer feedback, the **Samsung S24** is perceived as weaker in overall satisfaction due to recurring issues with performance under real-world stress.\n\nWhile some users complain about the iPhone\'s restrictive ecosystem or specific model flaws (e.g., iPhone 14 Pro battery), the feedback against Samsung points to a more fundamental failure in core functionality: reliability and sustained performance.\n\n---\n\n### **Proposed Change for Samsung:**\n\n**Implement an advanced, next-generation vapor chamber cooling system focused on sustained performance during intensive, multi-application use.**\n\nThis hardware enhancement directly addresses the top recurring customer pain point for Samsung devices: thermal throttling, overheating, and significant battery drain during real-world "workhorse" scenarios.\n\n#### **Evidence and Justification:**\n\nThe feedback indicates a clear pattern where users find Samsung phones impressive in features but lacking in reliability 